### imports

In [15]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%autoreload 2

In [17]:
from regex_to_NFA import NFA_CLASS,NFA
from NFA_to_DFA import DFA_CLASS,State,Edge
import json
from parser_classes import EPSILON

### Functions

In [18]:
def json_to_nfa(json_path: str) -> NFA:
    """
    This function is used to convert the JSON data to NFA
    params:
        json_path: str: Path to the JSON file
    return:
        NFA: NFA object
    """
    # read the JSON file
    with open(json_path, "r") as file:
        json_data = json.load(file)
    # create the NFA object
    nfa = NFA(None, None, [], [])
    # create the states
    states = []
    acceping_index = 0
    for state in json_data:
        if state == "startingState":
            continue
        if json_data[state]["isTerminatingState"] == True:
            acceping_index = int(state[1:])
        states.append(State(name=state))
    nfa.states = states
    # set the start state becuase the name of the starting states is the index of it inside the states array
    nfa.start = states[int(json_data["startingState"][1:])]
    # set the accept state
    nfa.accept = states[acceping_index]
    # create the transitions
    transitions = []
    for state in json_data:
        if state == "startingState":
            continue
        for transition in json_data[state]:
            if transition == "isTerminatingState":
                continue

            # check if the transition is epsilon or not
            if transition.startswith("epsilon"):
                transitions.append(
                    Edge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {EPSILON},
                    )
                )
            else:
                transitions.append(
                    Edge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {transition},
                    )
                )
    nfa.transitions = transitions
    return nfa

In [19]:
# getting the inputs from the tansitions in the nfa
def get_inputs(transitions):
    """_summary_
    This function is used to get the inputs from the transitions in the NFA
    Args:
        transitions (List): List of transitions in the NFA (List of Edges)
    Returns:
        List: List of unique inputs in the NFA
    """
    inputs = []
    for transition in transitions:
        temp_in = list(transition.characters)[0]
        if temp_in not in inputs and temp_in != EPSILON:
            inputs.append(temp_in)
    return inputs

### Regex to NFA

In [20]:
# regex = "[A-Za-z]+[0-9]*"
# regex = "ab*c+de?(f|g|h)|mr|n|[p-qs0-9]"
# regex = "ab"
regex = "(A|B)+"

In [37]:
nfa = NFA_CLASS(regex)

Error: 'str' object has no attribute 'get'
Error: Visualization failed
Error: 'str' object has no attribute 'items'
Error: Visualization failed


### NFA to DFA

In [22]:
# deserialize the json file to get the nfa
deserialized_nfa = json_to_nfa("./nfa.json")

In [23]:
inputs = get_inputs(deserialized_nfa.transitions)

In [24]:
dfa = DFA_CLASS()

In [25]:
dfa.subset_construction(deserialized_nfa, inputs)

Current State:  {State(name='S6'), State(name='S0'), State(name='S2'), State(name='S4')}
Input: A, Move: {State(name='S2'), State(name='S5'), State(name='S7'), State(name='S0'), State(name='S1'), State(name='S4'), State(name='S6')}
Input: B, Move: {State(name='S2'), State(name='S3'), State(name='S5'), State(name='S7'), State(name='S0'), State(name='S4'), State(name='S6')}
Current State:  {State(name='S2'), State(name='S5'), State(name='S7'), State(name='S0'), State(name='S1'), State(name='S4'), State(name='S6')}
Input: A, Move: {State(name='S2'), State(name='S5'), State(name='S7'), State(name='S0'), State(name='S1'), State(name='S4'), State(name='S6')}
Input: B, Move: {State(name='S2'), State(name='S3'), State(name='S5'), State(name='S7'), State(name='S0'), State(name='S4'), State(name='S6')}
Current State:  {State(name='S2'), State(name='S3'), State(name='S5'), State(name='S7'), State(name='S0'), State(name='S4'), State(name='S6')}
Input: A, Move: {State(name='S2'), State(name='S5'), 

In [26]:
dfa.print_dfa()

Start State:  S6,S0,S2,S4
Accepting States:  ['S2,S5,S7,S0,S1,S4,S6', 'S2,S3,S5,S7,S0,S4,S6']
Non Accepting States:  ['S6,S0,S2,S4']
States:  ['S6,S0,S2,S4', 'S2,S5,S7,S0,S1,S4,S6', 'S2,S3,S5,S7,S0,S4,S6']
Transitions: 
From: S6,S0,S2,S4, To: S2,S5,S7,S0,S1,S4,S6, Characters: {'A'}
From: S6,S0,S2,S4, To: S2,S3,S5,S7,S0,S4,S6, Characters: {'B'}
From: S2,S5,S7,S0,S1,S4,S6, To: S2,S5,S7,S0,S1,S4,S6, Characters: {'A'}
From: S2,S5,S7,S0,S1,S4,S6, To: S2,S3,S5,S7,S0,S4,S6, Characters: {'B'}
From: S2,S3,S5,S7,S0,S4,S6, To: S2,S5,S7,S0,S1,S4,S6, Characters: {'A'}
From: S2,S3,S5,S7,S0,S4,S6, To: S2,S3,S5,S7,S0,S4,S6, Characters: {'B'}


In [27]:
dfa.rename_dfa_states()
dfa.print_dfa()

Start State:  S0
Accepting States:  ['S1', 'S2']
Non Accepting States:  ['S0']
States:  ['S0', 'S1', 'S2']
Transitions: 
From: S0, To: S1, Characters: {'A'}
From: S0, To: S2, Characters: {'B'}
From: S1, To: S1, Characters: {'A'}
From: S1, To: S2, Characters: {'B'}
From: S2, To: S1, Characters: {'A'}
From: S2, To: S2, Characters: {'B'}


In [28]:
dfa.dfa_to_json("dfa.json")

In [29]:
dfa.visualize_dfa()

Error: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
Error: Visualization DFA failed


### Minimizing DFA

In [30]:
minimized_dfa = dfa.minimize_dfa()

In [31]:
# create a new object of DFA_CLASS to use the minimized dfa
minimized_dfa_obj = DFA_CLASS(minimized_dfa,inputs)

In [32]:
# print the minimized dfa
minimized_dfa_obj.print_dfa()

Start State:  S0
Accepting States:  ['S1', 'S2']
Non Accepting States:  ['S0']
States:  ['S0', 'S1', 'S2']
Transitions: 
From: S0, To: S1, Characters: {'A'}
From: S0, To: S2, Characters: {'B'}
From: S1, To: S1, Characters: {'A'}
From: S1, To: S2, Characters: {'B'}
From: S2, To: S1, Characters: {'A'}
From: S2, To: S2, Characters: {'B'}


In [33]:
minimized_dfa_obj.dfa_to_json("minimized_dfa.json")

In [34]:
minimized_dfa_obj.visualize_dfa(path="minimized_dfa.gv")

Error: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
Error: Visualization DFA failed


### Testing

In [35]:
input_regex = input("Enter the regular expression: ")
nfa = NFA_CLASS(input_regex)
deserialized_nfa = json_to_nfa("./nfa.json")
inputs = get_inputs(deserialized_nfa.transitions)
dfa = DFA_CLASS()
dfa.subset_construction(deserialized_nfa, inputs)
dfa.rename_dfa_states()
# dfa.print_dfa()
minimized_dfa = dfa.minimize_dfa()
minimized_dfa_obj = DFA_CLASS(minimized_dfa,inputs)
minimized_dfa_obj.rename_dfa_states()
# minimized_dfa_obj.print_dfa()
minimized_dfa_obj.dfa_to_json("minimized_dfa.json")
minimized_dfa_obj.visualize_dfa(path="minimized_dfa.gv")

Exception: No more tokens to parse